In [1]:
import torch
import random
import numpy as np
from itertools import chain
import torch.nn as nn
import itertools as it
import os, shutil
import pandas as pd
from pandas.core.common import flatten
from scipy.interpolate import make_interp_spline, interp1d
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from tqdm import tqdm
%cd GraphSSL

/home/shihan/GAT/GraphSSL


### Customized Dataset

In [11]:
random.seed(42)
t_threshold = 0.5  # in seconds
min_d_threshold = 10  # in meters
# t_prediction = 5 # in frames
danger_zone = 15 # in frames
sequence_len = 4 # in frames
sequence_dist = 40 # in frames

In [12]:
info_dir = '../csv_final/info/'   # directory of 'info' folder
all_info = []
for filename in os.listdir(info_dir):
    if filename.endswith('txt'):
        all_info.append(filename)
pedestrian = []
rear_end = []
switch_lane = [] # switch_lane
t_bone = []
opposite_front = []
opposite_merge = []
merging = []
other = []
for info in all_info:
    txt_file = open(info_dir+info, "r").read().split('\n') # split txt file with lines
    first_cls = txt_file[2].split(': ')[-1]
    second_cls = txt_file[9].split(': ')[-1]
    first_fault = txt_file[7].split(' ')[-1]
    second_fault = txt_file[14].split(' ')[-1]
    first_direction = txt_file[5].split(' ')[-1]
    second_direction = txt_file[12].split(' ')[-1]
    first_collision_side = txt_file[6].split(' ')[-1]
    second_collision_side = txt_file[13].split(' ')[-1]
    second_relative_direction_to_first = txt_file[15].split(' ')[-1]
    if first_cls == 'pedestrian' or second_cls == 'pedestrian':
        pedestrian.append(info)
    elif first_fault == 'rear_end' or second_fault == 'rear_end':
        rear_end.append(info)
    elif first_fault == 'switch_lane' or second_fault == 'switch_lane':
        switch_lane.append(info)
    elif first_direction == 'straight' and second_direction == 'straight':
        t_bone.append(info)
    elif second_relative_direction_to_first == 'opposite' and 'straight' in [first_direction, second_direction]:
        opposite_front.append(info)
    elif second_relative_direction_to_first == 'opposite':
        opposite_merge.append(info)
    elif first_fault == 'entrance_ramp' or second_fault == 'entrance_ramp':
        merging.append(info)
    elif second_relative_direction_to_first=='right' and first_direction=='left' and second_direction=='straight':
        merging.append(info)
    elif second_relative_direction_to_first=='left' and first_direction=='straight' and second_direction=='left':
        merging.append(info)
    elif second_relative_direction_to_first=='right' and first_direction=='straight' and second_direction=='right':
        merging.append(info)
    elif second_relative_direction_to_first=='left' and first_direction=='right' and second_direction=='straight':
        merging.append(info)
    else:
        other.append(info)
random.shuffle(pedestrian)
random.shuffle(rear_end)
random.shuffle(switch_lane)
random.shuffle(t_bone)
random.shuffle(opposite_front)
random.shuffle(opposite_merge)
random.shuffle(merging)
random.shuffle(other)
acc_all = {'pedestrian':len(pedestrian), 'rear_end':len(rear_end),
        'switch_lane':len(switch_lane), 't_bone':len(t_bone),
        'opposite_front':len(opposite_front), 'opposite_merge': len(opposite_merge),
        'merging':len(merging), 'other':len(other),
        'total': len(pedestrian+rear_end+switch_lane+t_bone+opposite_front+opposite_merge+merging+other)}
print(acc_all)
acc_train = {'pedestrian':math.ceil(len(pedestrian)*0.8), 'rear_end':math.ceil(len(rear_end)*0.8),
        'switch_lane':math.ceil(len(switch_lane)*0.8), 't_bone':math.ceil(len(t_bone)*0.8),
        'opposite_front':math.ceil(len(opposite_front)*0.8), 'opposite_merge': math.ceil(len(opposite_merge)*0.8),
        'merging':math.ceil(len(merging)*0.8), 'other':math.ceil(len(other)*0.8)}
acc_test = {'pedestrian':int(len(pedestrian)/5), 'rear_end':int(len(rear_end)/5),
        'switch_lane':int(len(switch_lane)/5), 't_bone':int(len(t_bone)/5),
        'opposite_front':int(len(opposite_front)/5), 'opposite_merge': int(len(opposite_merge)/5),
        'merging':int(len(merging)/5), 'other':int(len(other)/5)
        }
print('training dataset:')
print(acc_train)
print('testing dataset:')
print(acc_test)

{'pedestrian': 8, 'rear_end': 19, 'switch_lane': 13, 't_bone': 31, 'opposite_front': 22, 'opposite_merge': 19, 'merging': 40, 'other': 44, 'total': 196}
training dataset:
{'pedestrian': 7, 'rear_end': 16, 'switch_lane': 11, 't_bone': 25, 'opposite_front': 18, 'opposite_merge': 16, 'merging': 32, 'other': 36}
testing dataset:
{'pedestrian': 1, 'rear_end': 3, 'switch_lane': 2, 't_bone': 6, 'opposite_front': 4, 'opposite_merge': 3, 'merging': 8, 'other': 8}


In [13]:
accident_dir = '../csv_final/accident/'
normal_dir = '../csv_final/normal/'
train_scene_list = []
test_scene_list = []
accident_names = ['pedestrian','rear_end','switch_lane','t_bone','opposite_front','opposite_merge','merging','other']
all_names = accident_names+['no_accident']
major_acc_num = 36
accident_type_all = [pedestrian,rear_end,switch_lane,t_bone,opposite_front,opposite_merge,merging,other]
balanced_all_acc_train = []
for accident_type in accident_type_all:
    random.shuffle(accident_type)
    test_scene_list += accident_type[ : int(len(accident_type)/5)]
    train_num_accident_type = math.ceil(len(accident_type)*0.8)
    minor_class_multiplier = int(major_acc_num/train_num_accident_type)
    balanced_all_acc_train.append(accident_type[int(len(accident_type)/5):]*minor_class_multiplier 
                                  + accident_type[int(len(accident_type)/5):][:major_acc_num-math.ceil(len(accident_type)*0.8)*minor_class_multiplier])
for i in range(major_acc_num):
    for accident_type in balanced_all_acc_train:
        train_scene_list.append(accident_type[i])
#     train_scene_list += accident_type[int(len(accident_type)/5) : ]
normal_scene_list = []
for filename in os.listdir(normal_dir):
    if filename.endswith('csv'):
        normal_scene_list.append(filename)
random.shuffle(normal_scene_list)
# train_scene_list += normal_scene_list[len(test_scene_list) : len(test_scene_list)+len(train_scene_list)]
# test_scene_list += normal_scene_list[ : len(test_scene_list)]

In [15]:
crash_frame_ids = []
for filename in os.listdir(accident_dir):
    if filename.endswith('csv'):
        df = pd.read_csv(accident_dir+filename)
        crash_frame_id = min(df[df['acc_inv'] == 1]['frame_id'].tolist())
        if crash_frame_id < 70:
            print(crash_frame_id, filename)
        crash_frame_ids.append(crash_frame_id)
print(min(crash_frame_ids))

69 train_02_Town05_type001_subtype0002_scenario00031.csv
37 val_01_Town04_type001_subtype0002_scenario00025.csv
65 train_06_Town03_type001_subtype0001_scenario00020.csv
68 train_08_Town01_type001_subtype0001_scenario00012.csv
53 val_02_Town04_type001_subtype0001_scenario00036.csv
60 val_01_Town04_type001_subtype0001_scenario00039.csv
59 val_02_Town07_type001_subtype0002_scenario00027.csv
51 train_07_Town02_type001_subtype0001_scenario00014.csv
35 train_01_Town07_type001_subtype0002_scenario00036.csv
35 Mini_ds_Town10HD_type001_subtype0001_scenario00003.csv
40 val_02_Town05_type001_subtype0001_scenario00017.csv
45 train_01_Town10HD_type001_subtype0001_scenario00010.csv
52 val_01_Town01_type001_subtype0001_scenario00014.csv
47 train_08_Town04_type001_subtype0002_scenario00003.csv
41 train_04_Town07_type001_subtype0001_scenario00004.csv
42 train_08_Town05_type001_subtype0001_scenario00013.csv
49 train_02_Town10HD_type001_subtype0001_scenario00008.csv
56 train_05_Town07_type001_subtype0001

In [14]:
grouped_train = []  # (8, 36, 2*sequence_len)
for accident_class in balanced_all_acc_train:
    sub_class = []
    for scene in accident_class:
        if scene in normal_scene_list:
            y = 'no_accident'
            df = pd.read_csv(normal_dir+scene.replace('.txt','.csv'))  # read the i-th csv file
            crash_frame_id = df[-1:]['frame_id'].item()
            crash_ids = [314159, 314159]
        else:
            for accident_type in accident_type_all:
                if scene in accident_type:
                    y = accident_names[accident_type_all.index(accident_type)]
            df = pd.read_csv(accident_dir+scene.replace('.txt','.csv'))
            crash_frame_id = min(df[df['acc_inv'] == 1]['frame_id'].tolist())
            crash_ids = df[df['acc_inv'] == 1]['obj_id'].tolist() 
        frame_id_list = list(set(df['frame_id'].tolist()))  # all frame ids in a list
        # print(scene)
        edge_count = 0
        crash_pred_frame_id = crash_frame_id - 1
        if danger_zone+sequence_dist+danger_zone <= crash_pred_frame_id:
            pred_frame_id_list = list(range(crash_pred_frame_id-danger_zone-sequence_dist-danger_zone+1, 
                                            crash_pred_frame_id-danger_zone-sequence_dist+1)) + list(range(crash_pred_frame_id-danger_zone+1, 
                                                                                                           crash_pred_frame_id+1))
        else:
            pred_frame_id_list = list(range(0, danger_zone)) + list(range(crash_pred_frame_id-danger_zone+1, 
                                                                                                           crash_pred_frame_id+1))
        pair_seq = []
        for pred_frame_id in pred_frame_id_list:
            df_pred_frame = df[df['frame_id'] == pred_frame_id]
            pred_frame_id_list = df_pred_frame['obj_id'].tolist()  # all node ids in predicted frame
            sq_x_numeric = torch.tensor([]) # (# of nodes, 4, 6)
            for frame_id in list(range(pred_frame_id-sequence_len+1, pred_frame_id+1)):
                df_frame = df[df['frame_id'] == frame_id]
                df_numeric = df_frame[['x_center', 'y_center', 'bbox_x', 'bbox_y', 'vel_x', 'vel_y']]
                x_numeric = torch.tensor(df_numeric.to_numpy(), dtype=torch.float)
                id_list = df_frame['obj_id'].tolist()  # all node ids in predicted frame
                frame_x_numeric = torch.tensor([])  # (# of nodes, 6)
                for node_id in pred_frame_id_list:
                    if node_id in id_list:
                        frame_x_numeric = torch.cat([frame_x_numeric, x_numeric[id_list.index(node_id)].unsqueeze(0)], 0)
                    else:
                        frame_x_numeric = torch.cat([frame_x_numeric, torch.full([1, 6], torch.nan)], 0)
                sq_x_numeric = torch.cat([sq_x_numeric, frame_x_numeric.unsqueeze(1)], 1)
            sq_x_numeric_normed = sq_x_numeric / torch.max(sq_x_numeric)    # (# of nodes, 4, 6)
            df_dummies = pd.get_dummies(df_pred_frame[['obj_cls']].astype(pd.CategoricalDtype(categories=['car',
                                                                                                'van',
                                                                                                'truck',
                                                                                                'motorcycle',
                                                                                                'cyclist',
                                                                                                'pedestrian'
                                                                                                ])), dtype=float)
            node_idx_list = list(range(len(id_list)))
            edge_index = [[],[]]
            edge_features = []
            edge_id = []
            for node_connection in it.combinations(node_idx_list, 2):
                x1 = df_numeric['x_center'].tolist()[node_connection[0]]
                y1 = df_numeric['y_center'].tolist()[node_connection[0]]
                v_x1 = df_numeric['vel_x'].tolist()[node_connection[0]]
                v_y1 = df_numeric['vel_y'].tolist()[node_connection[0]]
                x2 = df_numeric['x_center'].tolist()[node_connection[1]]
                y2 = df_numeric['y_center'].tolist()[node_connection[1]]
                v_x2 = df_numeric['vel_x'].tolist()[node_connection[1]]
                v_y2 = df_numeric['vel_y'].tolist()[node_connection[1]]
                delta_x = x2 - x1
                delta_y = y2 - y1
                delta_v_x = v_x1 - v_x2
                delta_v_y = v_y1 - v_y2
                if delta_v_x**2+delta_v_y**2 == 0:
                    min_d = math.sqrt(delta_x**2+delta_y**2)
                    t_accident = 0
                else:
                    t_accident = (delta_x*delta_v_x+delta_y*delta_v_y)/(delta_v_x**2+delta_v_y**2)
                    if t_accident < 0:
                        min_d = math.sqrt(delta_x**2+delta_y**2)
                    else:
                        min_d = abs((delta_x*delta_v_y-delta_y*delta_v_x))/math.sqrt(delta_v_x**2+delta_v_y**2)
                if min_d <= min_d_threshold and t_accident <= t_threshold:
                    edge_index[0].append(node_connection[0])
                    edge_index[0].append(node_connection[1])
                    edge_index[1].append(node_connection[1])
                    edge_index[1].append(node_connection[0])
                    edge_features.append([t_accident, min_d])
                    edge_features.append([t_accident, min_d])
                    edge_id.append([id_list[node_connection[0]], id_list[node_connection[1]]])
                    edge_id.append([id_list[node_connection[1]], id_list[node_connection[0]]])
            edge_features_norm = nn.functional.normalize(torch.tensor(edge_features, dtype=torch.float), p=1.0, dim = 0)
            if pred_frame_id == crash_pred_frame_id:
                if crash_ids in edge_id:
                    print('Crash Edge Built!')
                else:
                    print('Crash Edge Not Built!')
            if pred_frame_id in list(range(crash_pred_frame_id-danger_zone+1, crash_pred_frame_id+1)):
                y = 1
            else:
                y = 0
            data = Data(edge_index = torch.tensor(edge_index, dtype=torch.long),
                x_numeric = sq_x_numeric_normed,
                x = torch.tensor(df_dummies.to_numpy(), dtype=torch.float),    # x_dummies
                edge_attr = edge_features_norm,
                y = torch.tensor([y], dtype=torch.long),
                frame_to_crash = torch.tensor(crash_frame_id-pred_frame_id, dtype=torch.float),
                )
            pair_seq.append(data)
        sub_class.append(pair_seq)
    grouped_train.append(sub_class)

Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Not Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Not Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Not Built!
Crash Edge Built!
Crash Edge Not Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Not Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Not Built!
Crash Edge Built!
Cras

In [15]:
paired_test = []
for scene in test_scene_list:
    if scene in normal_scene_list:
        y = 'no_accident'
        df = pd.read_csv(normal_dir+scene.replace('.txt','.csv'))  # read the i-th csv file
        crash_frame_id = df[-1:]['frame_id'].item()
        crash_ids = [314159, 314159]
    else:
        for accident_type in accident_type_all:
            if scene in accident_type:
                y = accident_names[accident_type_all.index(accident_type)]
        df = pd.read_csv(accident_dir+scene.replace('.txt','.csv'))
        crash_frame_id = min(df[df['acc_inv'] == 1]['frame_id'].tolist())
        crash_ids = df[df['acc_inv'] == 1]['obj_id'].tolist() 
    frame_id_list = list(set(df['frame_id'].tolist()))  # all frame ids in a list
    crash_pred_frame_id = crash_frame_id - 1
    if danger_zone+sequence_dist+danger_zone <= crash_pred_frame_id:
        pred_frame_id_list = list(range(crash_pred_frame_id-danger_zone-sequence_dist-danger_zone+1, 
                                        crash_pred_frame_id-danger_zone-sequence_dist+1)) + list(range(crash_pred_frame_id-danger_zone+1, 
                                                                                                        crash_pred_frame_id+1))
    else:
        pred_frame_id_list = list(range(0, danger_zone)) + list(range(crash_pred_frame_id-danger_zone+1, 
                                                                                                        crash_pred_frame_id+1))
    pair_seq = []
    for pred_frame_id in pred_frame_id_list:
        df_pred_frame = df[df['frame_id'] == pred_frame_id]
        pred_frame_id_list = df_pred_frame['obj_id'].tolist()  # all node ids in predicted frame
        sq_x_numeric = torch.tensor([]) # (# of nodes, 4, 6)
        for frame_id in list(range(pred_frame_id-sequence_len+1, pred_frame_id+1)):
            df_frame = df[df['frame_id'] == frame_id]
            df_numeric = df_frame[['x_center', 'y_center', 'bbox_x', 'bbox_y', 'vel_x', 'vel_y']]
            x_numeric = torch.tensor(df_numeric.to_numpy(), dtype=torch.float)
            id_list = df_frame['obj_id'].tolist()  # all node ids in predicted frame
            frame_x_numeric = torch.tensor([])  # (# of nodes, 6)
            for node_id in pred_frame_id_list:
                if node_id in id_list:
                    frame_x_numeric = torch.cat([frame_x_numeric, x_numeric[id_list.index(node_id)].unsqueeze(0)], 0)
                else:
                    frame_x_numeric = torch.cat([frame_x_numeric, torch.full([1, 6], torch.nan)], 0)
            sq_x_numeric = torch.cat([sq_x_numeric, frame_x_numeric.unsqueeze(1)], 1)
        sq_x_numeric_normed = sq_x_numeric / torch.max(sq_x_numeric)    # (# of nodes, 4, 6)
        df_dummies = pd.get_dummies(df_pred_frame[['obj_cls']].astype(pd.CategoricalDtype(categories=['car',
                                                                                            'van',
                                                                                            'truck',
                                                                                            'motorcycle',
                                                                                            'cyclist',
                                                                                            'pedestrian'
                                                                                            ])), dtype=float)
        node_idx_list = list(range(len(id_list)))
        edge_index = [[],[]]
        edge_features = []
        edge_id = []
        for node_connection in it.combinations(node_idx_list, 2):
            x1 = df_numeric['x_center'].tolist()[node_connection[0]]
            y1 = df_numeric['y_center'].tolist()[node_connection[0]]
            v_x1 = df_numeric['vel_x'].tolist()[node_connection[0]]
            v_y1 = df_numeric['vel_y'].tolist()[node_connection[0]]
            x2 = df_numeric['x_center'].tolist()[node_connection[1]]
            y2 = df_numeric['y_center'].tolist()[node_connection[1]]
            v_x2 = df_numeric['vel_x'].tolist()[node_connection[1]]
            v_y2 = df_numeric['vel_y'].tolist()[node_connection[1]]
            delta_x = x2 - x1
            delta_y = y2 - y1
            delta_v_x = v_x1 - v_x2
            delta_v_y = v_y1 - v_y2
            if delta_v_x**2+delta_v_y**2 == 0:
                min_d = math.sqrt(delta_x**2+delta_y**2)
                t_accident = 0
            else:
                t_accident = (delta_x*delta_v_x+delta_y*delta_v_y)/(delta_v_x**2+delta_v_y**2)
                if t_accident < 0:
                    min_d = math.sqrt(delta_x**2+delta_y**2)
                else:
                    min_d = abs((delta_x*delta_v_y-delta_y*delta_v_x))/math.sqrt(delta_v_x**2+delta_v_y**2)
            if min_d <= min_d_threshold and t_accident <= t_threshold:
                edge_index[0].append(node_connection[0])
                edge_index[0].append(node_connection[1])
                edge_index[1].append(node_connection[1])
                edge_index[1].append(node_connection[0])
                edge_features.append([t_accident, min_d])
                edge_features.append([t_accident, min_d])
                edge_id.append([id_list[node_connection[0]], id_list[node_connection[1]]])
                edge_id.append([id_list[node_connection[1]], id_list[node_connection[0]]])
        edge_features_norm = nn.functional.normalize(torch.tensor(edge_features, dtype=torch.float), p=1.0, dim = 0)
        if pred_frame_id == crash_pred_frame_id:
            if crash_ids in edge_id:
                print('Crash Edge Built!')
            else:
                print('Crash Edge Not Built!')
        if pred_frame_id in list(range(crash_pred_frame_id-danger_zone+1, crash_pred_frame_id+1)):
            y = 1
        else:
            y = 0
        data = Data(edge_index = torch.tensor(edge_index, dtype=torch.long),
            x_numeric = sq_x_numeric_normed,
            x = torch.tensor(df_dummies.to_numpy(), dtype=torch.float),    # x_dummies
            edge_attr = edge_features_norm,
            y = torch.tensor([y], dtype=torch.long),
            frame_to_crash = torch.tensor(crash_frame_id-pred_frame_id, dtype=torch.float),
            )
        pair_seq.append(data)
    paired_test.append(pair_seq)

Crash Edge Built!
Crash Edge Not Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Not Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!


In [32]:
dataset_test = []   # (70, 1)
test_scene_list += normal_scene_list[ : len(test_scene_list)]
for scene in test_scene_list:
    if scene in normal_scene_list:
        y = 0
        df = pd.read_csv(normal_dir+scene.replace('.txt','.csv'))  # read the i-th csv file
        crash_frame_id = df[-1:]['frame_id'].item()
        crash_ids = [314159, 314159]
    else:
        y = 1
        df = pd.read_csv(accident_dir+scene.replace('.txt','.csv'))
        crash_frame_id = min(df[df['acc_inv'] == 1]['frame_id'].tolist())
        crash_ids = df[df['acc_inv'] == 1]['obj_id'].tolist() 
    pred_frame_id = crash_frame_id - t_prediction
    # print(scene)
    df_pred_frame = df[df['frame_id'] == pred_frame_id]
    pred_frame_id_list = df_pred_frame['obj_id'].tolist()  # all node ids in predicted frame
    sq_x_numeric = torch.tensor([]) # (# of nodes, 4, 6)
    for frame_id in list(range(pred_frame_id-sequence_len+1, pred_frame_id+1)):
        df_frame = df[df['frame_id'] == frame_id]
        df_numeric = df_frame[['x_center', 'y_center', 'bbox_x', 'bbox_y', 'vel_x', 'vel_y']]
        x_numeric = torch.tensor(df_numeric.to_numpy(), dtype=torch.float)
        id_list = df_frame['obj_id'].tolist()  # all node ids in predicted frame
        frame_x_numeric = torch.tensor([])  # (# of nodes, 6)
        for node_id in pred_frame_id_list:
            if node_id in id_list:
                frame_x_numeric = torch.cat([frame_x_numeric, x_numeric[id_list.index(node_id)].unsqueeze(0)], 0)
            else:
                frame_x_numeric = torch.cat([frame_x_numeric, torch.full([1, 6], torch.nan)], 0)
        sq_x_numeric = torch.cat([sq_x_numeric, frame_x_numeric.unsqueeze(1)], 1)
    sq_x_numeric_normed = sq_x_numeric / torch.max(sq_x_numeric)    # (# of nodes, 4, 6)
    df_dummies = pd.get_dummies(df_pred_frame[['obj_cls']].astype(pd.CategoricalDtype(categories=['car',
                                                                                        'van',
                                                                                        'truck',
                                                                                        'motorcycle',
                                                                                        'cyclist',
                                                                                        'pedestrian'
                                                                                        ])), dtype=float)
    node_idx_list = list(range(len(id_list)))
    edge_index = [[],[]]
    edge_features = []
    edge_id = []
    for node_connection in it.combinations(node_idx_list, 2):
        x1 = df_numeric['x_center'].tolist()[node_connection[0]]
        y1 = df_numeric['y_center'].tolist()[node_connection[0]]
        v_x1 = df_numeric['vel_x'].tolist()[node_connection[0]]
        v_y1 = df_numeric['vel_y'].tolist()[node_connection[0]]
        x2 = df_numeric['x_center'].tolist()[node_connection[1]]
        y2 = df_numeric['y_center'].tolist()[node_connection[1]]
        v_x2 = df_numeric['vel_x'].tolist()[node_connection[1]]
        v_y2 = df_numeric['vel_y'].tolist()[node_connection[1]]
        delta_x = x2 - x1
        delta_y = y2 - y1
        delta_v_x = v_x1 - v_x2
        delta_v_y = v_y1 - v_y2
        if delta_v_x**2+delta_v_y**2 == 0:
            min_d = math.sqrt(delta_x**2+delta_y**2)
            t_accident = 0
        else:
            t_accident = (delta_x*delta_v_x+delta_y*delta_v_y)/(delta_v_x**2+delta_v_y**2)
            if t_accident < 0:
                min_d = math.sqrt(delta_x**2+delta_y**2)
            else:
                min_d = abs((delta_x*delta_v_y-delta_y*delta_v_x))/math.sqrt(delta_v_x**2+delta_v_y**2)
        if min_d <= min_d_threshold:
            edge_index[0].append(node_connection[0])
            edge_index[0].append(node_connection[1])
            edge_index[1].append(node_connection[1])
            edge_index[1].append(node_connection[0])
            edge_features.append([t_accident, min_d])
            edge_features.append([t_accident, min_d])
            edge_id.append([id_list[node_connection[0]], id_list[node_connection[1]]])
            edge_id.append([id_list[node_connection[1]], id_list[node_connection[0]]])
    edge_features_norm = nn.functional.normalize(torch.tensor(edge_features, dtype=torch.float), p=1.0, dim = 0)
    data = Data(edge_index = torch.tensor(edge_index, dtype=torch.long),
        x_numeric = sq_x_numeric_normed,
        x = torch.tensor(df_dummies.to_numpy(), dtype=torch.float),    # x_dummies
        edge_attr = edge_features_norm,
        y = torch.tensor([y], dtype=torch.long),
        
        )
    if crash_ids in edge_id:
        print('Crash Edge Built!')
    else:
        print('Crash Edge Not Built!')
    dataset_test.append(data)

Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Built!
Crash Edge Not Bui

### SSL Training

In [ ]:
args = {
    "device" : torch.device("cuda:3" if torch.cuda.is_available() else "cpu"),
    "save" : "ssl_model",
    "lr" : 0.001,
    "epochs" : 200,
    "batch_size" : 32,
    "num_workers" : 2,
    "model" : "gat", # choices are ["gcn", "gin", "resgcn", "gat", "graphsage", "sgc"]
    "input_dim" : 12, # input dimension for node features
    "feat_dim" : 128,
    "layers" : 3,
    "loss" : "infonce", # choices are ["infonce", "jensen_shannon"]
    "augment_list" : ['edge_perturbation', 'diffusion'],
    # choices are ["edge_perturbation", "diffusion", "diffusion_with_sample",
    # "node_dropping", "random_walk_subgraph", "node_attr_mask"]
    "train_data_percent" : 1,
}


class AttributeDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttributeDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

args = AttributeDict(args)

In [ ]:
import importlib, data_utils
importlib.reload(data_utils)

from data_utils import *

# split the data into train / val / test sets
train_dataset, val_dataset, test_dataset = split_dataset(dataset_SSL, args.train_data_percent)

# build_loader is a dataloader which gives a paired sampled - the original x and the positively
# augmented x obtained by applying the transformations in the augment_list as an argument
train_loader = build_loader(args, train_dataset, "train")
val_loader = build_loader(args, val_dataset, "val")
test_loader = build_loader(args, test_dataset, "test")

Dataset split: 288*2 70 70
Number of classes: 9


In [30]:
from models_SSL_June17 import *

# easy initialization of the GNN model encoder to map graphs to embeddings needed for contrastive training
model = Encoder(args.input_dim, args.feat_dim, n_layers=args.layers, gnn=args.model, pool="sum", bn=True, node_level=False, graph_level=True)
model = model.to(args.device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

In [31]:
from loss import *

def run(epoch, mode, dataloader):
	if mode == "train":
		model.train()
	elif mode == "val" or mode == "test":
		model.eval()

	contrastive_fn = eval(args.loss + "()")

	losses = []
	for data in dataloader:
		data.to(args.device)

		# readout_anchor is the embedding of the original datapoint x on passing through the model
		readout_anchor = model((data.x_anchor,
		                        data.edge_index_anchor, data.x_anchor_batch))

		# readout_positive is the embedding of the positively augmented x on passing through the model
		readout_positive = model((data.x_pos,
									data.edge_index_pos, data.x_pos_batch))

		# negative samples for calculating the contrastive loss is computed in contrastive_fn
		loss = contrastive_fn(readout_anchor, readout_positive)

		if mode == "train":
			# backprop
			optimizer.zero_grad()
			loss.backward()
			optimizer.step()

		# keep track of loss values
		losses.append(loss.item())

	# gather the results for the epoch
	epoch_loss = sum(losses) / len(losses)
	return epoch_loss


######################################################################

if not os.path.isdir(os.path.join("logs", args.save)):
    os.makedirs(os.path.join("logs", args.save))

best_train_loss, best_val_loss = float("inf"), float("inf")

for epoch in range(args.epochs):
    train_loss = run(epoch, "train", train_loader)
    val_loss = run(epoch, "val", val_loader)
    log = "Epoch {}, Train Loss: {:.3f}, Val Loss: {:.3f}"
    print(log.format(epoch, train_loss, val_loss))

    # save model
    is_best_loss = False
    if val_loss < best_val_loss:
        best_epoch, best_train_loss, best_val_loss, is_best_loss = \
                                            epoch, train_loss, val_loss, True

    model.save_checkpoint(os.path.join("logs", args.save), optimizer, epoch,
                          best_train_loss, best_val_loss, is_best_loss)

print("Train Loss at epoch {} (best model): {:.3f}".format(best_epoch, best_train_loss))
print("Val Loss at epoch {} (best model): {:.3f}".format(best_epoch, best_val_loss))


ValueError: not enough values to unpack (expected 4, got 3)

### Classifier Training

In [16]:
args = {
    "device" : torch.device("cuda:3" if torch.cuda.is_available() else "cpu"),
    "save" : "LSTM_Node_BC_Reg_model",
    "load" : "ssl_model",
    "lr" : 0.0005,
    "lr_edge" : 0.001,
    "epochs" : 200,
    "batch_size" : 8,
    "num_workers" : 2,
    "model" : "gat", # choices are ["gcn", "gin", "resgcn", "gat", "graphsage", "sgc"]
    "input_dim" : 12, # input dimension for node features
    # "num_classes" : len(all_names), # collision
    "num_classes" : 1,
    "feat_dim" : 8,
    "layers" : 3,
    "train_data_percent" : 0.8,
}

class AttributeDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttributeDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

args = AttributeDict(args)

In [17]:
grouped_train_dataset = grouped_train
val_dataset = list(chain.from_iterable(paired_test))
test_dataset = list(chain.from_iterable(paired_test))
# val_dataset = dataset_test
# test_dataset = test_dataset
val_loader = DataLoader(val_dataset, num_workers=args.num_workers, batch_size=30*args.batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, num_workers=args.num_workers, batch_size=30*args.batch_size, shuffle=True)

print("Dataset split: {} {} {}".format(str(len(grouped_train)*len(grouped_train[0]))+'*30', len(val_dataset), len(val_dataset)))
print("Number of classes: {}".format(args.num_classes))

Dataset split: 288*30 1050 1050
Number of classes: 1


In [18]:
import importlib, models_LSTM_Node_Reg_June24
importlib.reload(models_LSTM_Node_Reg_June24)

from models_LSTM_Node_Reg_June24 import *
# classification model is a GNN encoder followed by linear layer
model = GraphClassificationModel(args.input_dim, 
                                 args.feat_dim, 
                                 n_layers=args.layers, 
                                 gnn=args.model, 
                                 output_dim=args.num_classes,
                                 sequence_len = sequence_len
                                # , load=args.load
                                 )

for param in model.parameters():
    param.requires_grad = True

model = model.to(args.device)

optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

In [19]:
import warnings
warnings.filterwarnings("ignore")

model.train()
loss_fn = torch.nn.BCEWithLogitsLoss()
loss_mse = torch.nn.MSELoss()

is_best_loss = False
best_epoch, best_train_loss, best_val_loss, best_result = 0, float("inf"), float("inf"), float("inf")
best_train_acc, best_val_acc = 0.0, 0.0
best_train_reg_error, best_val_reg_error = 10.0, 10.0
for epoch in range(args.epochs):
    losses = []
    reg_losses = []
    correct = 0
    frame_error = 0
    for acc_type in grouped_train_dataset:
        random.shuffle(acc_type)
    train_dataset = []
    for i in range(major_acc_num):
        for acc_type in grouped_train_dataset:
            random.shuffle(acc_type[i])
            train_dataset += acc_type[i]
    flattened = train_dataset   # (15+15)*batch_size*36
    train_loader = DataLoader(flattened, num_workers=args.num_workers, batch_size=2*15*args.batch_size, shuffle=False)
    for data in tqdm(train_loader):
        data.to(args.device)
        data_input = data.x_numeric, data.x, data.edge_index, data.batch, data.edge_attr

        regression_score = model(data_input)[1][data.y == 1]
        regression_gt = data.frame_to_crash[data.y == 1]
        loss_reg = torch.sqrt(loss_mse(torch.flatten(regression_score, start_dim=0), regression_gt))/(danger_zone*10)
        frame_error += abs(torch.sub(torch.flatten(regression_score, start_dim=0), regression_gt)).sum().item() / regression_gt.size(0)
        reg_losses.append(loss_reg.item())
        optimizer.zero_grad()
        loss_reg.backward()
        optimizer.step()

        gt = data.y.float()
        scores = model(data_input)[0].squeeze(1)
        loss = loss_fn(scores, gt)
        pred = []
        for pr in scores:
            if torch.sigmoid(pr)>0.5:
                pred.append(1)
            else:
                pred.append(0)
        correct += int((torch.tensor(pred).to(args.device) == gt).sum()) / gt.size(0)
        losses.append(loss.item())
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss_train = sum(losses) / len(losses)
    epoch_reg_loss_train = sum(reg_losses) / len(reg_losses)
    epoch_acc_train = correct / len(train_loader)
    epoch_reg_error_train = frame_error / len(train_loader)


    # validation
    losses = []
    edge_losses = []
    reg_losses = []
    correct = 0
    edge_correct = 0
    frame_error = 0

    for data in val_loader:
        data.to(args.device)
        data_input = data.x_numeric, data.x, data.edge_index, data.batch, data.edge_attr
        labels = data.y
        gt = labels.float()
        scores = model(data_input)[0].squeeze(1)
        loss = loss_fn(scores, gt)
        pred = []
        for pr in scores:
            if torch.sigmoid(pr)>0.5:
                pred.append(1)
            else:
                pred.append(0)
        correct += int((torch.tensor(pred).to(args.device) == gt).sum()) / gt.size(0)
        losses.append(loss.item())
        regression_score = model(data_input)[1][data.y == 1]
        regression_gt = data.frame_to_crash[data.y == 1]
        loss_reg = torch.sqrt(loss_mse(torch.flatten(regression_score, start_dim=0), regression_gt))/(danger_zone*10)
        frame_error += abs(torch.sub(torch.flatten(regression_score, start_dim=0), regression_gt)).sum().item() / regression_gt.size(0)
        reg_losses.append(loss_reg.item())


    # gather the results for the epoch
    epoch_loss_val = sum(losses) / len(losses)
    epoch_acc_val = correct / len(val_loader)
    epoch_reg_loss_val = sum(reg_losses) / len(reg_losses)
    epoch_reg_error_val = frame_error / len(val_loader)


    log = "Epoch {},   Train Loss: {:.3f},            Train Accuracy: {:.3f},            Val Loss: {:.3f}, Val Accuracy: {:.3f}, \nTrain Regression Loss: {:.3f}, Train Regression Error: {:.3f}, Val Regression Loss: {:.3f}, Val Regression Error: {:.3f}\n"
    print(log.format(epoch, 
                     epoch_loss_train, epoch_acc_train, epoch_loss_val, epoch_acc_val, 
                     epoch_reg_loss_train, epoch_reg_error_train, epoch_reg_loss_val, epoch_reg_error_val))

    is_best_loss = False
    if epoch_reg_error_val-epoch_acc_val < best_result:
        best_epoch, best_train_loss, best_val_loss, is_best_loss = epoch, epoch_reg_loss_train, epoch_reg_loss_val, True
        best_train_acc, best_val_acc = epoch_acc_train, epoch_acc_val
        best_result = epoch_reg_error_val-epoch_acc_val
        best_train_reg_error, best_val_reg_error = epoch_reg_error_train, epoch_reg_error_val

print('best Epoch: ', best_epoch, ', best total train loss: ', best_train_loss, ', best total val loss: ', best_val_loss)
print('best train accuracy: ', best_train_acc, ', best validation accuracy: ', best_val_acc)
print('best train regression error: ', best_train_reg_error, ', best validation regression error: ', best_val_reg_error)

  0%|          | 0/36 [00:00<?, ?it/s]

100%|██████████| 36/36 [00:01<00:00, 23.15it/s]


Epoch 0,   Train Loss: 0.677,            Train Accuracy: 0.537,            Val Loss: 0.659, Val Accuracy: 0.536, 
Train Regression Loss: 0.054, Train Regression Error: 6.900, Val Regression Loss: 0.042, Val Regression Error: 5.145



100%|██████████| 36/36 [00:01<00:00, 23.53it/s]


Epoch 1,   Train Loss: 0.667,            Train Accuracy: 0.551,            Val Loss: 0.650, Val Accuracy: 0.597, 
Train Regression Loss: 0.036, Train Regression Error: 4.482, Val Regression Loss: 0.031, Val Regression Error: 3.939



...

Epoch 197,   Train Loss: 0.177,            Train Accuracy: 0.886,            Val Loss: 0.753, Val Accuracy: 0.739, 
Train Regression Loss: 0.025, Train Regression Error: 3.144, Val Regression Loss: 0.029, Val Regression Error: 3.607



100%|██████████| 36/36 [00:01<00:00, 23.25it/s]


Epoch 198,   Train Loss: 0.176,            Train Accuracy: 0.885,            Val Loss: 0.712, Val Accuracy: 0.724, 
Train Regression Loss: 0.025, Train Regression Error: 3.130, Val Regression Loss: 0.029, Val Regression Error: 3.620



100%|██████████| 36/36 [00:01<00:00, 23.99it/s]


Epoch 199,   Train Loss: 0.167,            Train Accuracy: 0.895,            Val Loss: 0.700, Val Accuracy: 0.757, 
Train Regression Loss: 0.025, Train Regression Error: 3.125, Val Regression Loss: 0.029, Val Regression Error: 3.630

best Epoch:  129 , best total train loss:  0.02635596863304575 , best total val loss:  0.027636028081178664
best train accuracy:  0.8689814814814816 , best validation accuracy:  0.7255555555555555
best train regression error:  3.3283089814362707 , best validation regression error:  3.4850224690407976


In [306]:
# best Epoch:  66 , best total train loss:  0.027757388694832724 , best total val loss:  0.028834787756204606
# best train accuracy:  0.8131944444444442 , best validation accuracy:  0.7558333333333334
# best train regression error:  3.5587899667245364 , best validation regression error:  3.6617657449655594

# best Epoch:  111 , best total train loss:  0.02667617068315546 , best total val loss:  0.028392944484949112
# best train accuracy:  0.8563657407407408 , best validation accuracy:  0.7791666666666666
# best train regression error:  3.413623908714012 , best validation regression error:  3.6083966161638052

# best Epoch:  76 , best total train loss:  0.02778545239319404 , best total val loss:  0.028726327046751977
# best train accuracy:  0.8261574074074075 , best validation accuracy:  0.7424999999999999
# best train regression error:  3.5680612776014535 , best validation regression error:  3.6783903061792342

0.66616819302241

In [42]:
best_epoch, best_train_loss, best_val_loss = model.load_checkpoint(os.path.join("logs", args.save), optimizer)
model.eval()

losses = []
edge_losses = []
reg_losses = []
correct = 0
edge_correct = 0
frame_error = 0

for data in test_loader:
    data.to(args.device)

    data_input = data.x, data.edge_index, data.batch, data.edge_attr

    labels = data.y

    gt = []
    for label in labels:
        if label == 0:
            gt.append([1,0])
        else:
            gt.append([0,1])
    gt = torch.tensor(gt, dtype=torch.float32).to(args.device)


    # get class scores from model
    scores = model(data_input)[0]

    loss = loss_fn(scores, gt)

    # Keep track of loss and accuracy
    pred = scores.argmax(dim=1)
    correct += int((pred == labels).sum()) / labels.size(0)
    losses.append(loss.item())


    batch_node_label = []
    for data_point in data.batch:
        if data.y[data_point] == 0:
            batch_node_label.append(False)
        else:
            batch_node_label.append(True)
    batch_node_label = torch.tensor(batch_node_label, dtype=torch.bool).to(args.device)
    batch_edge_label = batch_node_label[data.edge_index[0]]
    edge_score = model(data_input)[1][batch_edge_label]
    gt_edge = []
    batch_edge_index = data.obj_ids[data.edge_index[0]][batch_edge_label]
    for i in range(edge_score.size(0)):
        if batch_edge_index[i] in data.crash_ids:
            gt_edge.append([0,1])
        else:
            gt_edge.append([1,0])
    gt_edge = torch.tensor(gt_edge, dtype=torch.float32).to(args.device)
    loss_edge = loss_fn(edge_score, gt_edge)
    edge_losses.append(loss_edge.item())
    pred_edge = edge_score.argmax(dim=1)
    labels_edge = gt_edge.argmax(dim=1)
    edge_correct += int((pred_edge == labels_edge).sum()) / labels_edge.size(0)


    regression_score = model(data_input)[2][data.y == 1]
    regression_gt = data.frame_to_crash[data.y == 1]
    loss_reg = torch.sqrt(loss_mse(torch.flatten(regression_score, start_dim=0), regression_gt))/t_prediction
    frame_error += abs(torch.sub(torch.flatten(regression_score, start_dim=0), regression_gt)).sum().item() / regression_gt.size(0)
    reg_losses.append(loss_reg.item())

# gather the results for the epoch
loss_test = sum(losses) / len(losses)
correct_test = correct / len(test_loader)
epoch_edge_loss_test = sum(edge_losses) / len(edge_losses)
epoch_edge_acc_test = edge_correct / len(test_loader)
epoch_reg_loss_test = sum(reg_losses) / len(reg_losses)
epoch_reg_error_test = frame_error / len(test_loader)

print("Test Loss at epoch {}: {:.3f}, Test Accuracy: {:.3f}, \nTest Edge Loss: {:.3f}, Test Edge Accuracy: {:.3f} \nTest Regression Loss: {:.3f}, Test Regression Error: {:.3f}".format(best_epoch, loss_test, correct_test, epoch_edge_loss_test, epoch_edge_acc_test, epoch_reg_loss_test, epoch_reg_error_test))

Test Loss at epoch 99: 2.376, Test Accuracy: 0.720, 
Test Edge Loss: 0.604, Test Edge Accuracy: 0.821 
Test Regression Loss: 0.301, Test Regression Error: 7.256
